In [1]:
!pip install praw unidecode pandas openpyxl

In [4]:
import praw
import pandas as pd
from datetime import datetime
import time
import logging

logging.getLogger("praw").setLevel(logging.ERROR)

# Load Product Names from Excel
file_path = "/content/Headphones.xlsx"
sheet_name = "Headphone-Avail-Price"

# Read product names from the specified sheet
df_products = pd.read_excel(file_path, sheet_name=sheet_name)

# Ensure "Product Name" column exists
if "Product Name" not in df_products.columns:
    raise ValueError(" 'Product Name' column not found in the Excel sheet.")

# Extract product names
products = df_products["Product Name"].dropna().unique().tolist()

#  Reddit API Credentials (Ensure these are correct and not shared publicly)
reddit = praw.Reddit(
    client_id='jNAMMjqIULpTFdVMVeVxkg',
    client_secret='S5lIYbBLN0WDcoKvoYUgKzF3p2gB7g',
    user_agent='MyShopt by /u/TemporaryTop5884'
)

def get_all_comments_from_top_posts(search_query, min_comments=100, comments_per_post=None, wait_time=3):

    all_comments = []
    total_comments = 0

    while total_comments < min_comments:
        #  Fetch top Reddit posts for the query
        submissions = list(reddit.subreddit("all").search(
            search_query,
            sort="relevance",
            syntax="lucene",
            time_filter="all",
            limit=100  # Fetch up to 100 posts per query
        ))

        if not submissions:
            break

        #  Process each submission
        for submission in submissions:

            # Load full submission with comments
            full_submission = reddit.submission(id=submission.id)
            full_submission.comments.replace_more(limit=5)  # Expand comments
            flat_comments = full_submission.comments.list()

            #  Apply per-post comment limit (if set)
            if comments_per_post:
                flat_comments = flat_comments[:comments_per_post]

            #  Store comments
            for comment in flat_comments:
                all_comments.append({
                    "product": search_query,
                    "post_title": submission.title,
                    "post_id": submission.id,
                    "post_url": f"https://www.reddit.com{submission.permalink}",
                    "post_score": submission.score,
                    "post_created_utc": datetime.fromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                    "subreddit": str(submission.subreddit),

                    "comment_id": comment.id,
                    "comment_author": str(comment.author),
                    "comment_body": comment.body,
                    "comment_score": comment.score,
                    "comment_created_utc": datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                    "comment_url": f"https://www.reddit.com{comment.permalink}",
                    "comment_depth": comment.depth,
                    "is_top_level": comment.depth == 0
                })
                total_comments += 1

                # Stop if we hit the minimum required comments
                if total_comments >= min_comments:
                    break

            if total_comments >= min_comments:
                break

        # Avoid rate limits
        time.sleep(wait_time)

    return pd.DataFrame(all_comments)

# Fetch & Save Data for Each Product
all_products_comments = []

#  Fetch Data for Each Product
for product in products:
    search_query = f"Review {product} headphone"
    comments_df = get_all_comments_from_top_posts(search_query)

    if not comments_df.empty:
        all_products_comments.append(comments_df)
    else:
        print(f" No comments found for {product}.")

#  Combine all product comments into a single DataFrame
if all_products_comments:
    combined_df = pd.concat(all_products_comments, ignore_index=True)

    # Save as a single Excel file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"All_Headphone_Reviews_Reddit_Comments_{timestamp}.xlsx"
    combined_df.to_excel(filename, index=False)

    # Optionally save each product as a separate sheet in the same Excel file
    with pd.ExcelWriter(f"Headphone_Reviews_By_Product_{timestamp}.xlsx") as writer:
        for product in products:
            product_df = combined_df[combined_df['product'] == f"Review {product} headphone"]
            if not product_df.empty:
                # Create sheet name (Excel has 31 character limit for sheet names)
                sheet_name = product[:31]
                product_df.to_excel(writer, sheet_name=sheet_name, index=False)
else:
    print("\n No comments found for any products.")

print("\n All comments saved.")



 All comments saved.


In [2]:
!pip uninstall numpy gensim -y


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: gensim 4.3.3
Uninstalling gensim-4.3.3:
  Successfully uninstalled gensim-4.3.3


In [3]:
!pip install --upgrade numpy

  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.


In [4]:
!pip install --upgrade gensim



  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4


In [5]:
!pip show gensim

Name: gensim
Version: 4.3.3
Summary: Python framework for fast Vector Space Modelling
Home-page: https://radimrehurek.com/gensim/
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: LGPL-2.1-only
Location: /usr/local/lib/python3.11/dist-packages
Requires: numpy, scipy, smart-open
Required-by: 


In [6]:
!python --version

Python 3.11.11


In [13]:
import os
import glob
import re
import nltk
import string
import numpy as np
import pandas as pd
from datetime import datetime
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# Ensure required NLTK data packages are available
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/stopwords')
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt')
    nltk.download('stopwords')
    nltk.download('punkt_tab')

# Define a set of stop words for filtering common words in text
stop_words = set(stopwords.words('english'))
cached_embeddings = {}  # Cache embeddings to avoid redundant computations

# File and sheet details for product specifications
headphone_file = "/content/Headphones.xlsx"
sheet_name = "Headphone-Avail-Price"

def safe_string(text):
   ## Converts input text to a string safely, returning an empty string if the input is NaN.

    if pd.isna(text):
        return ""
    return str(text)

def contains_product_name(text, title, product_name):
    ## Checks if the product name appears in either the text or the title. It also checks for partial matches of the product name components.

    text = safe_string(text).lower()
    title = safe_string(title).lower()
    product_name = safe_string(product_name).lower()

    if product_name in text or product_name in title:
        return True

    product_parts = product_name.split()
    # Ensure at least one match is required (can adjust threshold if needed)
    required_matches = max(1, len(product_parts) // 1)

    text_matches = sum(1 for part in product_parts
                      if len(part) > 2 and part not in stop_words and part in text)

    title_matches = sum(1 for part in product_parts
                       if len(part) > 2 and part not in stop_words and part in title)

    return (text_matches >= required_matches) or (title_matches >= required_matches)

def extract_key_terms(description):
    ## Extracts key terms from the product description by tokenizing, converting to lower case, and removing punctuation and stop words.

    text = safe_string(description).lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = word_tokenize(text)
    key_terms = [word for word in words if word.isalnum()
                and word not in stop_words
                and len(word) > 2]
    return set(key_terms)

def extract_feature_phrases(text):
    ## Extracts potential feature phrases from the product description.
    # It splits the text by newlines and applies various heuristics to capture important features and phrases.

    text = safe_string(text)
    lines = text.split('\n')
    features = []

    for line in lines:
        line = line.strip()
        if not line:
            continue
        # Remove list markers or numbers at the start of the line
        line = re.sub(r'^[•\-\*\d]+\.?\s*', '', line)
        if ':' in line:
            parts = line.split(':', 1)
            feature = parts[0].strip()
            if feature and len(feature.split()) <= 5:
                features.append(feature.lower())
            features.append(line.lower())
        else:
            if len(line.split()) <= 10:
                features.append(line.lower())

    # If few features are detected, try extracting key terms and bigrams from combined text
    if len(features) < 3 and len(lines) >= 3:
        combined_text = ' '.join(lines)
        tokens = word_tokenize(combined_text.lower())
        important_terms = [word for word in tokens if word.isalnum() and word not in stop_words]
        important_bigrams = [f"{tokens[i]} {tokens[i+1]}" for i in range(len(tokens)-1)
                           if tokens[i].isalnum() and tokens[i+1].isalnum()
                           and tokens[i] not in stop_words]
        features.extend(important_terms[:10])
        features.extend(important_bigrams[:5])

    return list(set(features))

def compute_feature_overlap(comment, product_features):
    ## Calculates a simple overlap score between the comment and product features.

    comment_text = safe_string(comment).lower()
    matches = 0
    for feature in product_features:
        if feature in comment_text:
            matches += 1
            continue
        feature_parts = feature.split()
        if any(part in comment_text for part in feature_parts if len(part) > 3 and part not in stop_words):
            matches += 0.5
            continue
    return matches

def get_embedding(text, model):

    ## Retrieves the embedding for the given text using the provided model.

    text = safe_string(text)
    if text in cached_embeddings:
        return cached_embeddings[text]
    embedding = model.encode(text)
    cached_embeddings[text] = embedding
    return embedding

def is_generic_or_bot_comment(text):
    ## Checks if a comment is generic or appears to be generated by a bot.

    text = safe_string(text).lower()

    # Check if the comment mentions a bot
    if re.search(r'\bbot\b', text) is not None:
        return True

    # Patterns that indicate transaction tracking or generic bot messages
    transaction_patterns = [
        r'this comment is now being tracked',
        r'your flair will update',
        r'please reply to the above comment with your feedback',
        r'once you reply, you will both get credit',
        r'if you did \*\*not\*\* complete a transaction',
        r'thank you!\s*---\s*\[\^'
    ]

    for pattern in transaction_patterns:
        if re.search(pattern, text, re.IGNORECASE):
            return True

    return False

def is_quality_comment(comment, min_words=15, min_sentences=2):
    ## Determines if a comment meets quality criteria based on its word count and sentence structure.
    #Removes URLs and formatting symbols before evaluation.

    comment = safe_string(comment)
    if not comment:
        return False
    # Remove URLs from the comment
    comment_text = re.sub(r'https?://\S+|www\.\S+', '', comment)
    # Remove common markdown symbols and punctuation that may interfere with sentence tokenization
    comment_text = re.sub(r'[*_~`#\[\]\(\)\{\}>]', '', comment_text)
    words = [w for w in word_tokenize(comment_text.lower())
             if w.isalnum() and w not in stop_words]
    sentences = sent_tokenize(comment_text)
    return (len(words) >= min_words and
            len(sentences) >= min_sentences and
            any(len(s.split()) > 5 for s in sentences))

def product_focused_similarity(product_description, comment, product_name, product_features, model):
    ## Computes a similarity score between a product's description and a comment.

    product_description = safe_string(product_description)
    comment = safe_string(comment)
    product_name = safe_string(product_name)
    # Calculate how many product features overlap with the comment text
    feature_overlap = compute_feature_overlap(comment, product_features)
    # Get embeddings for semantic similarity measurement
    prod_embedding = get_embedding(product_description, model)
    comment_embedding = get_embedding(comment, model)
    semantic_sim = cosine_similarity([prod_embedding], [comment_embedding])[0][0]
    # Check if the product name is mentioned in the comment (full or partial bonus)
    product_mention = 1 if product_name.lower() in comment.lower() else 0.5
    final_score = (0.4 * semantic_sim +
                  0.4 * min(1.0, feature_overlap / max(1, len(product_features) / 3)) +
                  0.2 * product_mention)
    return {
        'final_score': final_score,
        'semantic_similarity': semantic_sim,
        'feature_overlap': feature_overlap,
        'product_mention': product_mention
    }

def score_and_rank_comments(df, product_description, product_name, model, batch_size=100):
    ## Filters and ranks comments related to a product.

    # Exclude comments from AutoModerator and generic/bot comments
    if 'comment_author' in df.columns:
        df = df[
            (df['comment_author'] != 'AutoModerator') &
            (~df['comment_body'].apply(lambda x: is_generic_or_bot_comment(x)))
        ].copy()
    else:
        return pd.DataFrame()

    # Helper to safely retrieve values from a DataFrame row
    def safe_get(row, column, default=""):
        if column in row and not pd.isna(row[column]):
            return row[column]
        return default

    # Label comments that are of sufficient quality
    df['is_quality'] = df.apply(
        lambda row: (
            is_quality_comment(safe_get(row, 'comment_body')) and
            not is_generic_or_bot_comment(safe_get(row, 'comment_body'))
        ),
        axis=1
    )
    filtered_df = df[df['is_quality']].copy()
    if filtered_df.empty:
        return pd.DataFrame()

    # Filter comments that mention the product name in text or title
    filtered_df['mentions_product'] = filtered_df.apply(
        lambda row: contains_product_name(
            safe_get(row, 'comment_body'),
            safe_get(row, 'post_title'),
            product_name
        ),
        axis=1
    )

    product_mention_df = filtered_df[filtered_df['mentions_product']].copy()
    if product_mention_df.empty:
        return pd.DataFrame()

    # Extract key terms and features from the product description
    key_terms = extract_key_terms(product_description)
    product_features = extract_feature_phrases(product_description)

    total_rows = len(product_mention_df)
    scores = []

    # Process comments in batches to show progress using tqdm
    for i in tqdm(range(0, total_rows, batch_size), desc=f"Processing {product_name}"):
        batch = product_mention_df.iloc[i:min(i+batch_size, total_rows)]
        batch_scores = [
            product_focused_similarity(
                product_description,
                safe_get(row, 'comment_body'),
                product_name,
                product_features,
                model
            ) for _, row in batch.iterrows()
        ]
        scores.extend(batch_scores)

    score_df = pd.DataFrame(scores)
    result_df = pd.concat([product_mention_df.reset_index(drop=True), score_df], axis=1)
    result_df = result_df.sort_values(by='final_score', ascending=False)
    # result_df = result_df[result_df['final_score'] > 0.3]

    # Mark whether the product is mentioned in the post title
    result_df['product_in_title'] = result_df.apply(
        lambda row: product_name.lower() in safe_string(row.get('post_title', '')).lower() or
                   any(part in safe_string(row.get('post_title', '')).lower()
                      for part in product_name.lower().split()
                      if len(part) > 2 and part not in stop_words),
        axis=1
    )

    # Mark whether the product is mentioned in the comment body
    result_df['product_in_body'] = result_df.apply(
        lambda row: product_name.lower() in safe_string(row.get('comment_body', '')).lower() or
                   any(part in safe_string(row.get('comment_body', '')).lower()
                      for part in product_name.lower().split()
                      if len(part) > 2 and part not in stop_words),
        axis=1
    )

    return result_df

def main():
    """
    Main routine:
    1. Loads the sentence transformer model.
    2. Finds the latest comments file.
    3. Reads product specifications.
    4. Scores and ranks comments per product.
    5. Outputs the top 5 unique comments per product into an Excel file.
    """
    model = SentenceTransformer('all-MiniLM-L6-v2')

    # Locate the most recent comments file
    possible_files = glob.glob("All_Headphone_Reviews_Reddit_Comments_*.xlsx")
    if not possible_files:
        return

    latest_file = max(possible_files, key=os.path.getmtime)

    try:
        comments_df = pd.read_excel(latest_file)
    except Exception as e:
        return

    # Check for required columns; rename if necessary
    required_columns = ['comment_body', 'post_title', 'comment_author']
    missing_columns = [col for col in required_columns if col not in comments_df.columns]
    if missing_columns:
        column_mapping = {}
        for req_col in missing_columns:
            potential_matches = [col for col in comments_df.columns
                                if req_col.lower() in col.lower() or
                                col.lower() in req_col.lower()]
            if potential_matches:
                column_mapping[req_col] = potential_matches[0]
        if column_mapping:
            comments_df = comments_df.rename(columns={v: k for k, v in column_mapping.items()})

    try:
        # Read product specifications from the Excel file
        spec_df = pd.read_excel("Headphones.xlsx", sheet_name="Headphone-Avail-Price")
        if not {'Product Name', 'Specifications'}.issubset(spec_df.columns):
            return
        products = dict(zip(spec_df['Product Name'].astype(str), spec_df['Specifications'].astype(str)))
    except Exception as e:
        return

    ranked_comments = {}
    # Process comments for each product
    for pn in products.keys():
        ranked_df = score_and_rank_comments(comments_df, products[pn], pn, model)
        if len(ranked_df) > 0:
            ranked_comments[pn] = ranked_df
        else:
            ranked_comments[pn] = pd.DataFrame()

    summary_data = []
    # For each product, select the top 5 unique comments based on the final score
    for pn, df in ranked_comments.items():
        if not df.empty:
            unique_comments = df.drop_duplicates(subset='comment_body')
            top_comments = unique_comments.sort_values(by='final_score', ascending=False).head(5)
            top_comments['product_name'] = pn
            summary_data.append(top_comments)

    if summary_data:
        all_ranked_top_comments = pd.concat(summary_data)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"top5_unique_comments_per_product_{timestamp}.xlsx"
        all_ranked_top_comments.to_excel(filename, index=False)

if __name__ == "__main__":
    main()


Processing Grado GW100x: 100%|██████████| 1/1 [00:00<00:00,  8.06it/s]
